In [ ]:
# Dependencies
import requests
import json
from pprint import pprint
import pandas as pd
import os
import csv
import numpy as np
import time 

path = os.path.join("Team_5/Gazetteer_2017_counties.csv")
GazCo = pd.read_csv(path)
GazCo.head(1)

#### Make me the correct url
url = 'https://geo.fcc.gov/api/census/block/find?'


In [ ]:

output_list = pd.DataFrame(columns=['FIPS','longitude','latitude'])

d= {'LAT': ['38.8816'], 'LON': ['-77.0910']}
df_geo = pd.DataFrame(data=d)
df_geo.head()

In [ ]:
df_geo = df_geo[['INTPTLAT','INTPTLONG']]
entry = 0

for (idx, row) in df_geo.iterrows():
    try:
        entry += 1 
        latitude = (row.loc['INTPTLAT'])
        longitude = (row.loc['INTPTLONG'])
        query_url = f"{url}&latitude={latitude}&longitude={longitude}&format=json"
        code = requests.get(query_url).json()
        county_fips = code["County"]["FIPS"]
        block_fips = code["Block"]["FIPS"]
        construction_list.at[idx, 'latitude'] = latitude
        construction_list.at[idx, 'longitude'] = longitude
        construction_list.at[idx, 'FIPS'] = county_fips
        construction_list.at[idx, 'FIPS_block'] = block_fips         
        print(entry)
    except:
        time.sleep(5)
        print("restarting...")
        pass